In [1]:
#OLLAMAPool WebUI Monitoring
import os
EndPoint_NodeStatus=os.environ.get('EndPoint_NodeStatus')

#Assert if the environment variables are set
if EndPoint_NodeStatus is None:
    raise ValueError("EndPoint_NodeStatus is not set")
    
    

In [2]:
#Trimmed down NodeStatus class
import json
class NodeStatus():
    
    def from_json(self,json_str):
        self.__dict__=json.loads(json_str)

In [3]:
#Main Message Handling Loop
import time
from typing import List
from azure.servicebus import ServiceBusClient, ServiceBusMessage

# Replace with your connection string and queue name
queue_name = "node-status"
hosts_Status={}

# Create a Service Bus client
servicebus_client = ServiceBusClient.from_connection_string(conn_str=EndPoint_NodeStatus)

def receive_messages_from_queue():
    # Create a receiver for the queue
    with servicebus_client.get_queue_receiver(queue_name=queue_name) as receiver:
        print("Receiving messages from the queue...")
        
        # Receive messages in a batch, you can specify max_message_count as needed
        received_msgs = receiver.receive_messages(max_message_count=20, max_wait_time=30)

        for msg in received_msgs:
            # Print the message payload
            print(f"Received message: {str(msg)}")
            
            #Update node status
            nodeStatus=NodeStatus()
            nodeStatus.from_json(str(msg))
            hosts_Status[nodeStatus.Host]=nodeStatus
            print(f"Hosts Status: {hosts_Status}")
            #print(nodeStatus.to_json())
            
            # Accept the message to remove it from the queue
            receiver.complete_message(msg)
            

#receive_messages_from_queue()


In [4]:
#Web Dash
from flask import Flask, render_template
from threading import Thread
import json

app = Flask(__name__)

# Example hosts_Status dictionary
# hosts_Status = {
#     "host1": {"Host": "host1", "OllamaHost": "ollama1", "Status": "Online", "Message": "All good", "Models": ["model1", "model2"], "LastQueryTime": 1234567890},
#     "host2": {"Host": "host2", "OllamaHost": "ollama2", "Status": "Offline", "Message": "Error", "Models": ["model3"], "LastQueryTime": 1234567890}
# }

@app.route('/')
def show_hosts():
    return render_template('hosts.html', hosts_status=hosts_Status)

# Thread to run Flask in Jupyter
def run_app():
    app.run(host='0.0.0.0', port=5000)

# Running Flask in a separate thread
flask_thread = Thread(target=run_app)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.102:5000
Press CTRL+C to quit


In [5]:
while True:
    receive_messages_from_queue()

Receiving messages from the queue...
Received message: {"Host": "Euclid", "OllamaHost": "http://localhost:11434", "Status": "Initializing...", "Message": "", "Models": ["llama3.1:latest"], "LastQueryTime": 0}
Hosts Status: {'Euclid': <__main__.NodeStatus object at 0x000001B0572DD460>}
Received message: {"Host": "Euclid", "OllamaHost": "http://localhost:11434", "Status": "Ready", "Message": "Waiting for Queries", "Models": ["llama3.1:latest"], "LastQueryTime": 0}
Hosts Status: {'Euclid': <__main__.NodeStatus object at 0x000001B0572DDA30>}
Received message: {"Host": "Euclid", "OllamaHost": "http://localhost:11434", "Status": "Running", "Message": "Processingdcbac0e5-dbc0-42cf-ad3a-9f2285bc7d8c", "Models": ["llama3.1:latest"], "LastQueryTime": 0}
Hosts Status: {'Euclid': <__main__.NodeStatus object at 0x000001B0572DDA60>}
Received message: {"Host": "Euclid", "OllamaHost": "http://localhost:11434", "Status": "Finsihed", "Message": "Processingdcbac0e5-dbc0-42cf-ad3a-9f2285bc7d8c", "Models":

192.168.1.102 - - [23/Sep/2024 20:37:50] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:37:50] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.102 - - [23/Sep/2024 20:37:51] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:37:53] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:37:54] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:37:56] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:37:57] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:37:58] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:37:59] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:38:01] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:38:01] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:38:02] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:38:02] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:38:03] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/2024 20:38:03] "GET / HTTP/1.1" 200 -
192.168.1.102 - - [23/Sep/202